In [1]:
"""
Get 100ms updates to bid/asks on a token from KuCoin exchange
@authors: Jimmy Singh 
"""

import asyncio
from IPython.display import clear_output
from kucoin.client import WsToken
from kucoin.ws_client import KucoinWsClient
import nest_asyncio
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import time

nest_asyncio.apply()

token = 'ETH-USDT'
depth = 50

async def main():
    async def deal_msg(msg):
        if msg['topic'] == f'/spotMarket/level2Depth{depth}:{token}':
            
            # print(f'Get ETH level3:{msg["data"]}\n')
            ab = msg["data"]

            asks = {}
            for a in ab['asks']:
                asks[float(a[0])] = np.round(float(a[1]),3)
            bids = {}
            for b in ab['bids']:
                bids[float(b[0])] = np.round(float(b[1]),3)
            
            asks_ps = list(asks.keys())
            asks_ps_size = list(asks.values())
            asks_spread = np.round(asks_ps[-1] - asks_ps[0],3)
            bids_ps = list(bids.keys())
            bids_ps_size = list(bids.values())
            bids_spread = np.round(bids_ps[0] - bids_ps[-1],3)
            
            bid_ask_spread = np.round(asks_ps[0] - bids_ps[0],3)
            avg_ask = np.round(np.sum(np.multiply(asks_ps,asks_ps_size))/np.sum(asks_ps_size),2)
            avg_bid = np.round(np.sum(np.multiply(bids_ps,bids_ps_size))/np.sum(bids_ps_size),2)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x= asks_ps, y=asks_ps_size,
                                name = f'asks', marker_color='red'))
            fig.add_trace(go.Bar(x= bids_ps, y=bids_ps_size,
                                name = f'bids', marker_color='green'))
            fig.update_layout(title={'text' : f'50 Bid/Asks Level 2 data of {token}',
                                'y':0.92,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis_title="Price",yaxis_title= f'Size', 
                                xaxis = {'rangeslider': {'visible': False}}, yaxis={'side': 'left'}, showlegend=True, width = 1200,height = 500)
            fig.show()


            print(f'Bid\Ask\Size:\n{np.round(bids_ps[0],2)} @ {np.round(asks_ps[0],2)}, {np.round(bids_ps_size[0],2)} by {np.round(asks_ps_size[0],2)}')
            print(f'Spread:{bid_ask_spread}\n')
            print(f'50 Bids\Asks Spread:\n{bids_spread} | {asks_spread}\n')
            print(f'Avg. Bid\Ask:\n{avg_bid} | {avg_ask} | Spread:{np.abs(np.round(avg_bid-avg_ask,2))}')

            # time.sleep(0.25)
            clear_output(wait=True)
    
    client = WsToken()
    ws_client = await KucoinWsClient.create(None, client, deal_msg, private = False)
    await ws_client.subscribe(f'/spotMarket/level2Depth{depth}:{token}')
    
    while True:
        await asyncio.sleep(60)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Bid\Ask\Size:
1595.66 @ 1595.67, 0.79 by 3.51
Spread:0.01

50 Bids\Asks Spread:
1.79 | 1.28

Avg. Bid\Ask:
1594.69 | 1596.32 | Spread:1.63
